In [2]:
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import warnings
import re
import ast

warnings.filterwarnings('ignore')


In [15]:
lift_df = pd.read_csv("lifts.csv", index_col=0)
data_for_lift = pd.read_csv("data_for_lift.csv", index_col=0)

In [17]:
lift_df

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>T')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'G>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '5', 'TERT', 'G>A')",0.00
Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '12', 'KRAS', 'C>T')",0.00
...,...,...,...
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.166666667, '25398284.0-25398284.0', 12.0, '...",0.00
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.233333333, '25398284.0-25398284.0', 12.0, '...",0.63
Soft Tissue Sarcoma,"('TMB (nonsynonymous)', 'Position', 'Protein_p...","(0.1, '55259515.0-55259515.0', 858.0, 'cTg/cGg...",0.00


In [17]:
# try_df = data_for_lift[data_for_lift["Cancer Type"] == "Colorectal Carcinoma"]
# try_df = data_for_lift[data_for_lift['Smoke Status'] == "Unknown"]
# try_df = try_df[try_df['SNP_event'] == "C>G"]
# try_df = try_df[try_df['Exon_Number'] == "02/05"]
# try_df = try_df[try_df['Protein_position'] == 12.0]
# try_df = try_df[try_df['Codons'] == "Ggt/Cgt"]
#
# set(try_df.index)
# try_df

,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,...,Extrahepatic Cholangiocarcinoma,Gallbladder Carcinoma,Gastric Cancer,Intrahepatic Cholangiocarcinoma,Liver Hepatocellular Carcinoma,Non Small Cell Lung Cancer,Pancreatic Cancer,Small Cell Lung Cancer,Soft Tissue Sarcoma,Position
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
Patient0257,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,61-70,Unknown,0.133333,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient0295,Non Small Cell Lung Cancer,Large Cell Lung Carcinoma,IV,Metastasis,Male,51-60,Unknown,1.866667,KRAS,12,...,0,0,0,0,0,1,0,0,0,25398285.0-25398285.0
Patient0378,Intrahepatic Cholangiocarcinoma,Intrahepatic Cholangiocarcinoma,Unknown,Primary,Male,51-60,Unknown,0.233333,KRAS,12,...,0,0,0,1,0,0,0,0,0,25398285.0-25398285.0
Patient0557,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,71-80,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient0582,Breast Carcinoma,Breast Invasive Carcinoma,IV,Metastasis,Female,41-50,Unknown,1.200000,KRAS,12,...,0,0,0,0,0,0,0,0,0,25398285.0-25398285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient8524,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Male,51-60,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient8538,Pancreatic Cancer,Pancreatic Adenocarcinoma,I,Primary,Male,41-50,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient8546,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Female,61-70,Unknown,0.100000,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0


In [18]:
lift_df.sort_values(by="Lift Value", ascending=False).head(1000)

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', 12.0, 'Ggt/Cgt')",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'Consequence',...","('Unknown', 'KRAS', 'missense_variant', 12.0, ...",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', '25398285.0-2539828...",27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...","('Unknown', 'KRAS', 'C>G', 'Ggt/Cgt', 'Substit...",27.03
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'SNP_event', 'P...","('Unknown', '12', 'C>G', '25398285.0-25398285....",27.03
...,...,...,...
Pancreatic Cancer,"('Smoke Status', 'Consequence', 'Exon_Number',...","('Unknown', 'missense_variant', '02/05', '51-6...",16.64
Pancreatic Cancer,"('Chromosome', 'Hugo_Symbol', 'Exon_Number', '...","('12', 'KRAS', '02/05', 0.133333333, 12.0)",16.64
Pancreatic Cancer,"('Smoke Status', 'SNP_event', 'Diagnosis Age',...","('Unknown', 'C>A', '51-60', '25398284.0-253982...",16.64


In [139]:
df = lift_merged.copy()

In [140]:
df.reset_index(inplace=True)

In [141]:
df.sort_values(by="Lift Value", ascending=False)

,Cancer Type,Feature Combination,Feature,Lift Value
12652,Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '12', 'KRAS', 'C>G', 'missense_var...",27.03
717,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'A>G', 'm...",26.22
25,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'A>G', 'm...",26.22
69,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', 'missense...",23.73
159,Breast Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Female', 'Unknown', '3', 'PIK3CA', '21/21', ...",23.73
...,...,...,...,...
6283,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '4', 'FAT1', 'Truncation')",0.00
6278,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '16', 'PLCG2', 'Substituti...",0.00
6268,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Smoker', '8', 'FAM135B', 'Substituti...",0.00
6262,Gallbladder Carcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '10', 'RET', 'Substitution...",0.00


In [142]:
# Improved function to safely parse tuple-like strings
def safe_eval(val):
    if pd.isna(val) or val == 'None' or val == 'nan':
        return None

    try:
        # Handle the case where it's already a proper string representation of a tuple
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        try:
            # Fix missing quotes around words and try again
            fixed_val = re.sub(r'(\b\w+\b)(?=\s*,|\s*\))', r"'\1'", val)
            fixed_val = re.sub(r'\(\s*,', '(', fixed_val)  # Fix empty first item
            fixed_val = re.sub(r',\s*\)', ')', fixed_val)  # Fix empty last item
            return ast.literal_eval(fixed_val)
        except (ValueError, SyntaxError):
            # If still fails, try to extract content between parentheses
            match = re.search(r'\((.*)\)', val)
            if match:
                items = match.group(1).split(',')
                # Clean and convert items
                processed_items = []
                for item in items:
                    item = item.strip()
                    if item and item != 'None':
                        # Try to convert to appropriate type
                        try:
                            if item.lower() == 'true':
                                processed_items.append(True)
                            elif item.lower() == 'false':
                                processed_items.append(False)
                            elif item.replace('.', '', 1).isdigit():
                                if '.' in item:
                                    processed_items.append(float(item))
                                else:
                                    processed_items.append(int(item))
                            else:
                                processed_items.append(item.strip("'\""))
                        except:
                            processed_items.append(item.strip("'\""))

                return tuple(processed_items) if processed_items else None
            return None

# Apply parsing with tqdm progress bar
tqdm.pandas(desc="Parsing Features")
df["Feature"] = df["Feature"].progress_apply(lambda x: safe_eval(str(x)) if not pd.isna(x) else None)
df["Feature Combination"] = df["Feature Combination"].progress_apply(lambda x: safe_eval(str(x)) if not pd.isna(x) else None)

Parsing Features: 100%|██████████| 15864/15864 [00:00<00:00, 136858.83it/s]


In [143]:
# Find all unique features across the dataset
all_features = set()
for features in df["Feature Combination"].dropna():
    if isinstance(features, tuple):  # Make sure it's actually a tuple
        all_features.update(features)

# Prepare transformed data
expanded_data = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    new_row = {
        "Cancer Type": row["Cancer Type"],
        "Lift Value": row["Lift Value"]
    }

    # The bug is here: Check if both are tuples AND verify they're the same length
    if isinstance(row["Feature Combination"], tuple) and isinstance(row["Feature"], tuple):
        if len(row["Feature Combination"]) == len(row["Feature"]):
            new_row.update(dict(zip(row["Feature Combination"], row["Feature"])))
        else:
            # Handle case where tuples are different lengths
            # Only use the matching portions
            min_length = min(len(row["Feature Combination"]), len(row["Feature"]))
            new_row.update(dict(zip(row["Feature Combination"][:min_length],
                                    row["Feature"][:min_length])))

    # Assign NaN to missing feature columns
    for feature in all_features:
        if feature not in new_row:
            new_row[feature] = None  # Using None to represent NaN

    # Only append rows that have at least one non-null feature value
    if any(new_row.get(feature) is not None for feature in all_features):
        expanded_data.append(new_row)
    else:
        # Optionally include even empty rows if that's what you want
        expanded_data.append(new_row)

# Convert to DataFrame
df_transformed = pd.DataFrame(expanded_data)

Processing Rows: 100%|██████████| 15864/15864 [00:00<00:00, 66896.33it/s]


In [217]:
# d = df_transformed[df_transformed["Protein_position"] == 249]
# d[d["Cancer Type"] == "Intrahepatic Cholangiocarcinoma"]
df_transformed = df_transformed.drop_duplicates()
df_transformed = df_transformed[df_transformed["Lift Value"] > 6]
df_transformed

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous)
25,Breast Carcinoma,26.22,Female,Unknown,3,PIK3CA,A>G,missense_variant,21/21,None,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,NaN
29,Breast Carcinoma,8.13,Female,Unknown,3,PIK3CA,G>A,missense_variant,10/21,None,178936091.0-178936091.0,545.0,Gag/Aag,Substitution/Indel,NaN
53,Breast Carcinoma,6.15,Female,Unknown,17,TP53,None,missense_variant,08/11,None,None,NaN,None,Substitution/Indel,NaN
62,Breast Carcinoma,11.16,Female,Unknown,3,PIK3CA,None,missense_variant,10/21,None,None,NaN,None,Substitution/Indel,NaN
69,Breast Carcinoma,23.73,Female,Unknown,3,PIK3CA,None,missense_variant,21/21,None,None,NaN,None,Substitution/Indel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15543,Soft Tissue Sarcoma,6.32,Male,Unknown,17,NF1,None,None,None,None,None,NaN,None,Truncation,NaN
15834,Soft Tissue Sarcoma,6.43,Female,Unknown,None,None,None,missense_variant,None,51-60,None,NaN,None,Substitution/Indel,0.100000
15841,Soft Tissue Sarcoma,6.62,Male,Unknown,None,None,None,missense_variant,None,51-60,None,NaN,None,Substitution/Indel,0.066667
15842,Soft Tissue Sarcoma,6.62,Female,Unknown,None,None,None,missense_variant,None,41-50,None,NaN,None,Substitution/Indel,0.100000


In [218]:
# def merge_rows(df):
#     # Step 1: First do the original grouping by exact Cancer Type and Lift Value
#     grouped = df.groupby(['Cancer Type', 'Lift Value'])
#
#     initial_merged_rows = []
#
#     for (cancer_type, lift_value), group in grouped:
#         # Start with the first row as the merged row
#         merged_row = group.iloc[0].copy()
#
#         # Go through all columns except 'Cancer Type' and 'Lift Value'
#         for col in df.columns:
#             if col not in ['Cancer Type', 'Lift Value']:
#                 # Fill in values from other rows in the group
#                 for i in range(1, len(group)):
#                     current_val = merged_row[col]
#                     new_val = group.iloc[i][col]
#
#                     # If current value is NaN and new value is not, use new value
#                     if pd.isna(current_val) and not pd.isna(new_val):
#                         merged_row[col] = new_val
#
#         initial_merged_rows.append(merged_row)
#
#     # Step 2: Create a dataframe from the initial merging
#     initial_merged_df = pd.DataFrame(initial_merged_rows)
#
#     # Step 3: Now merge rows with lift value differences < 1
#     # Sort by Cancer Type and Lift Value
#     df_sorted = initial_merged_df.sort_values(['Cancer Type', 'Lift Value']).reset_index(drop=True)
#
#     final_result_rows = []
#     processed_indices = set()
#
#     for i in range(len(df_sorted)):
#         if i in processed_indices:
#             continue
#
#         cancer_type = df_sorted.iloc[i]['Cancer Type']
#         lift_value = df_sorted.iloc[i]['Lift Value']
#
#         # Find all rows with same cancer type and lift value within threshold
#         similar_rows = []
#         for j in range(len(df_sorted)):
#             if j in processed_indices:
#                 continue
#
#             if (df_sorted.iloc[j]['Cancer Type'] == cancer_type and
#                 abs(df_sorted.iloc[j]['Lift Value'] - lift_value) < 3):
#                 similar_rows.append(j)
#                 processed_indices.add(j)
#
#         # Create a group DataFrame
#         group = df_sorted.iloc[similar_rows]
#
#         # Start with the first row as the merged row
#         merged_row = group.iloc[0].copy()
#
#         # Go through all columns except 'Cancer Type' and 'Lift Value'
#         for col in df_sorted.columns:
#             if col not in ['Cancer Type', 'Lift Value']:
#                 # Fill in values from other rows in the group
#                 for i in range(1, len(group)):
#                     current_val = merged_row[col]
#                     new_val = group.iloc[i][col]
#
#                     # If current value is NaN and new value is not, use new value
#                     if pd.isna(current_val) and not pd.isna(new_val):
#                         merged_row[col] = new_val
#
#         final_result_rows.append(merged_row)
#
#     return pd.DataFrame(final_result_rows)
#
# # Apply the function to your dataframe
# merged_df = merge_rows(df_transformed)

# def merge_rows(df):
#     feature_cols = [col for col in df.columns if col not in ['Cancer Type', 'Lift Value']]
#     grouped = df.groupby(['Cancer Type', 'Lift Value'])
#     initial_merged_rows = []
#
#     for (cancer_type, lift_value), group in grouped:
#         # Count number of feature columns with all non-null values equal
#         common_count = 0
#         for col in feature_cols:
#             non_null_vals = group[col].dropna().unique()
#             if len(non_null_vals) == 1 and len(non_null_vals) > 0:
#                 common_count += 1
#
#         if common_count < 2:
#             continue  # Skip merging if less than 2 common values
#
#         merged_row = group.iloc[0].copy()
#         for col in feature_cols:
#             non_null_vals = group[col].dropna().unique()
#             if len(non_null_vals) == 1:
#                 merged_row[col] = non_null_vals[0]
#             else:
#                 merged_row[col] = np.nan
#         initial_merged_rows.append(merged_row)
#
#     initial_merged_df = pd.DataFrame(initial_merged_rows)
#     # (Continue with your second merging step as before...)
#     return initial_merged_df
#

def merge_rows(df):
    feature_cols = [col for col in df.columns if col not in ['Cancer Type', 'Lift Value']]
    result_rows = []
    used = set()
    df = df.sort_values(['Cancer Type', 'Lift Value'], ascending=[True, False]).reset_index(drop=True)

    for cancer_type, group in df.groupby('Cancer Type'):
        group = group.reset_index()  # preserve original index
        n = len(group)

        for i in range(n):
            idx_i = group.loc[i, 'index']
            if idx_i in used:
                continue

            base_row = group.loc[i]
            merge_candidates = [i]

            for j in range(n):
                if i == j:
                    continue

                idx_j = group.loc[j, 'index']
                if idx_j in used:
                    continue

                comp_row = group.loc[j]

                # Check Lift Value diff
                # if abs(base_row['Lift Value'] - comp_row['Lift Value']) >= 3:
                #     continue

                # Count matching non-null features
                matches = 0
                consistent = True
                for col in feature_cols:
                    val1 = base_row[col]
                    val2 = comp_row[col]
                    if pd.notna(val1) and pd.notna(val2):
                        if val1 != val2:
                            consistent = False
                            break
                        matches += 1

                if consistent and matches >= 3:
                    merge_candidates.append(j)

            merge_indices = [group.loc[k, 'index'] for k in merge_candidates]
            if len(merge_candidates) > 1:
                subset = group.loc[merge_candidates]
                merged_row = base_row.copy()
                merged_row['Lift Value'] = subset['Lift Value'].max()
                for col in feature_cols:
                    non_nulls = subset[col].dropna()
                    merged_row[col] = non_nulls.iloc[0] if not non_nulls.empty else np.nan
                result_rows.append(merged_row.drop(labels='index'))
                used.update(merge_indices)
            else:
                # No merge: keep the original row
                result_rows.append(base_row.drop(labels='index'))
                used.add(idx_i)

    return pd.DataFrame(result_rows)


merged_df = merge_rows(df_transformed)

In [219]:
merged_df.sort_values(by=["Lift Value", "Cancer Type"], ascending=[False, True])
# df_transformed.sort_values(by=["Lift Value", "Cancer Type"], ascending=[False, True])

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous)
0,Pancreatic Cancer,27.03,Female,Unknown,12,KRAS,C>G,missense_variant,02/05,51-60,25398285.0-25398285.0,12.0,Ggt/Cgt,Substitution/Indel,0.066667
0,Breast Carcinoma,26.22,Female,Unknown,3,PIK3CA,A>G,missense_variant,21/21,41-50,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,0.100000
0,Small Cell Lung Cancer,20.05,Male,Smoker,13,RB1,None,None,None,None,None,NaN,None,Truncation,NaN
0,Intrahepatic Cholangiocarcinoma,16.57,Female,Unknown,2,IDH1,NaN,missense_variant,04/10,61-70,NaN,132.0,NaN,Substitution/Indel,0.100000
3,Pancreatic Cancer,16.25,Male,Unknown,12,KRAS,C>A,missense_variant,02/05,51-60,25398284.0-25398284.0,12.0,gGt/gTt,Substitution/Indel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Extrahepatic Cholangiocarcinoma,6.09,Male,Unknown,12,KRAS,C>A,missense_variant,NaN,61-70,NaN,NaN,NaN,Substitution/Indel,NaN
78,Pancreatic Cancer,6.09,Female,Unknown,9,CDKN2A,None,None,None,None,None,NaN,None,Substitution/Indel,NaN
8,Esophageal Carcinoma,6.08,Male,Unknown,9,NOTCH1,C>T,missense_variant,None,None,None,NaN,None,Substitution/Indel,NaN
6,Extrahepatic Cholangiocarcinoma,6.06,Female,Unknown,None,None,None,missense_variant,None,61-70,None,NaN,None,Substitution/Indel,0.133333


In [220]:
def row_to_sentence(row):
    return " AND ".join(
        f"{col} value is {val}"
        for col, val in row.items()
        if pd.notnull(val) and col not in {"Cancer Type", "Lift Value", "hypo_factors"}
    )

In [221]:
merged_df['hypo_factors'] = merged_df.apply(row_to_sentence, axis=1)
merged_df.rename(columns={'Lift Value': 'support'}, inplace=True)

In [222]:
merged_df

,Cancer Type,support,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous),hypo_factors
0,Breast Carcinoma,26.22,Female,Unknown,3,PIK3CA,A>G,missense_variant,21/21,41-50,178952085.0-178952085.0,1047.0,cAt/cGt,Substitution/Indel,0.100000,Sex value is Female AND Smoke Status value is ...
11,Breast Carcinoma,12.17,Female,Unknown,7,KMT2C,NaN,frameshift_variant,NaN,NaN,NaN,NaN,NaN,Truncation,NaN,Sex value is Female AND Smoke Status value is ...
12,Breast Carcinoma,11.65,Female,Unknown,17,TP53,C>T,missense_variant,06/11,41-50,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...
13,Breast Carcinoma,11.48,Female,Unknown,3,PIK3CA,G>A,missense_variant,10/21,NaN,178936082.0-178936082.0,542.0,Gaa/Aaa,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...
18,Breast Carcinoma,10.48,Female,Unknown,10,PTEN,None,None,None,None,None,NaN,None,Truncation,NaN,Sex value is Female AND Smoke Status value is ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,Small Cell Lung Cancer,6.12,Male,Unknown,13,RB1,None,frameshift_variant,None,None,None,NaN,None,Truncation,NaN,Sex value is Male AND Smoke Status value is Un...
0,Soft Tissue Sarcoma,14.73,Male,Unknown,4,KIT,NaN,missense_variant,NaN,41-50,NaN,NaN,NaN,Substitution/Indel,0.066667,Sex value is Male AND Smoke Status value is Un...
1,Soft Tissue Sarcoma,9.40,Female,Unknown,3,CTNNB1,A>G,missense_variant,03/15,41-50,41266124.0-41266124.0,41.0,Acc/Gcc,Substitution/Indel,0.100000,Sex value is Female AND Smoke Status value is ...
9,Soft Tissue Sarcoma,6.33,Female,Unknown,5,TERT,G>A,upstream_gene_variant,NaN,NaN,NaN,NaN,NaN,Substitution/Indel,NaN,Sex value is Female AND Smoke Status value is ...


In [223]:
merged_df.to_csv("models_hypotheses/LIFT_hypotheses_as_sentences.csv")

In [76]:
merged_df.to_csv("data_for_lift_merged.csv")

In [134]:
lift_merged = pd.read_csv("lifts_merged.csv", index_col=0)

In [136]:
lift_merged.sort_values(by="Lift Value", ascending=False)

np.int64(0)

In [71]:
tt = merged_df[merged_df["Cancer Type"] == "Intrahepatic Cholangiocarcinoma"]
# tt = tt[tt["Hugo_Symbolmbol"] == "KRAS"]
# tt = tt[tt["Smoke Status"] == "Unknown"]
# tt = tt[tt["Chromosome"] == "12"]
# tt = tt[tt["VAR_TYPE_SX"] == 'Substitution/Indel']
tt = tt[tt["Protein_position"] == 249]
tt

,Cancer Type,Lift Value,Sex,Smoke Status,Chromosome,Hugo_Symbol,SNP_event,Consequence,Exon_Number,Diagnosis Age,Position,Protein_position,Codons,VAR_TYPE_SX,TMB (nonsynonymous)
71,Intrahepatic Cholangiocarcinoma,3.06,Male,Unknown,17,TP53,C>A,missense_variant,07/11,41-50,7577534.0-7577534.0,249.0,agG/agT,Substitution/Indel,NaN


In [ ]:
('Female', 'Unknown', '12', 'KRAS', 'Substitution/Indel')

In [49]:
l = lift_merged[lift_merged['Feature'].astype(str).str.contains('249', na=False)]
l = l[l.index == "Intrahepatic Cholangiocarcinoma"]
l

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Intrahepatic Cholangiocarcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A', 'miss...",3.06
Intrahepatic Cholangiocarcinoma,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '17', 'TP53', 'missense_variant', ...",2.88
Intrahepatic Cholangiocarcinoma,"('Sex', 'Smoke Status', 'Chromosome', 'Hugo_Sy...","('Male', 'Unknown', '17', 'TP53', 'C>A', 'miss...",3.06
Intrahepatic Cholangiocarcinoma,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...","('Unknown', '17', 'TP53', 'C>A', 'missense_var...",3.16


In [22]:
df = lift_merged.copy()
df.reset_index(inplace=True)

In [39]:
df[df["Lift Value"] == 27.03]

,Cancer Type,Feature Combination,Feature,Lift Value
12652,Pancreatic Cancer,"(Smoke Status, Chromosome, Hugo_Symbol, SNP_ev...","(Unknown, 12, KRAS, C>G, missense_variant, 02/...",27.03


In [77]:
df_new = pd.read_csv("lifts_second_round.csv", index_col=0)

In [78]:
df_new

,Cancer Type,Feature Combination,Feature,Lift Value
0,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Male', '12', 'KRAS', 'C>T', 'missense_varian...",0.00
1,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Female', '7', 'EGFR', 'T>G', 'missense_varia...",0.00
2,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Female', '12', 'KRAS', 'C>T', 'missense_vari...",0.00
3,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Male', '12', 'KRAS', 'C>A', 'missense_varian...",0.00
4,Breast Carcinoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'SNP_even...","('Male', '17', 'TP53', 'C>T', 'missense_varian...",0.00
...,...,...,...,...
8331979,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Female', '1', 'TIE1', 'missense_variant', 'S...",0.00
8331980,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Male', '5', 'SDHA', 'missense_variant', 'Sub...",0.83
8331981,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Male', 'X', 'RBM10', 'stop_gained', 'Truncat...",0.00
8331982,Soft Tissue Sarcoma,"('Sex', 'Chromosome', 'Hugo_Symbol', 'Conseque...","('Male', 'X', 'RBM10', 'frameshift_variant', '...",0.84
